## Load in data and remove extra columns ##

In [7]:
import pandas as pd
import glob
import os

from datetime import datetime

path = os.getcwd()
csv_files = glob.glob(os.path.join(path, "*.csv"))

for file in csv_files:
    df = pd.read_csv(file)

df = df[['Date', 'Last', 'First', 'Score', 'Sleep Hours']]

## Add rows with week averages ##

In [8]:
dates = df['Date'].unique()
weeks = []
week = []

# Separate days into weeks
for idx, date in enumerate(dates):
    
    # Create datetime object from str
    clean_date = datetime.strptime(date, "%Y-%m-%d")
    
    # Add day to current week
    week.append(date)
    
    # End of week (Monday)
    if clean_date.weekday() == 0 or idx == len(dates) - 1:
        weeks.append(week)
        week = []

# Add row with team averages for each full week
for week in weeks:
    score_mean = df.loc[df['Date'].isin(week), 'Score'].mean()
    sleep_mean = df.loc[df['Date'].isin(week), 'Sleep Hours'].mean()
    row = ({
        'Date': f"{week[0]} Weekly Avg",
        'Last': ' Team',
        'First': 'Average',
        'Score': score_mean,
        'Sleep Hours': sleep_mean
    })
    idx = df.loc[df['Date'] == week[-1]].index.array[-1] + 1
    df2 = pd.DataFrame(row, index=[idx])
    df = pd.concat([df, df2]).sort_index().reset_index(drop=True)
df['Score'] = df['Score'].round(2)
df['Sleep Hours'] = df['Sleep Hours'].round(1)
    



## Add conditional formatting and export to excel ##

In [9]:
score = 'Score'
sleep = 'Sleep Hours'

# Create multiindex dataframe
df = df.pivot_table(index=['Last', 'First'], columns=['Date'], values=['Score', 'Sleep Hours']).sort_index(axis=1, level='Date', ascending=False)

# Create individual dataframes for each week
wks = []
wk = pd.DataFrame()
for idx, col in enumerate(df):
    if idx > 0 and col[0] == 'Sleep Hours' and 'W' in col[1]:
        wk = wk.iloc[:, ::-1]
        wk.columns = pd.MultiIndex.from_tuples(wk.columns, names=['','Date']).swaplevel(0, 1)
        wks.append(wk)
        wk = pd.DataFrame()
    wk[col] = df[col]

# Score conditional formatting
def score_f(val):
    highlight = ''
    if val < 60:
        highlight = 'background-color: red'
    elif val >= 60 and val < 70:
        highlight = 'background-color: orange'
    elif val >= 70 and val < 85:
        highlight = 'background-color: yellow'
    elif val >= 85:
        highlight =  'background-color: #15DE00'
    return highlight

# Sleep conditional formatting
def sleep_f(val):
    highlight = ''
    if val < 6:
        highlight = 'background-color: red'
    elif val >= 6 and val < 8:
        highlight = 'background-color: yellow'
    elif val >= 8:
        highlight =  'background-color: #15DE00'
    return highlight

# Blank cells
def blank(val):
    highlight = ''
    if val == 'nan':
        highlight = 'backgroundcolor: white'
    return highlight

current_date = datetime.today().strftime('%Y-%m-%d')
with pd.ExcelWriter(f'WSoccer Readiness {current_date}.xlsx', engine='xlsxwriter') as writer:
    for idx, w in enumerate(wks):
        styled_w = (w
        .style
        .applymap(score_f, subset=[col for col in w.columns if col[1] == 'Score'])
        .applymap(sleep_f, subset=[col for col in w.columns if col[1] == 'Sleep Hours'])
        .applymap(blank)
        .set_properties(**{'text-align': 'center'})
         )
        styled_w.to_excel(writer, freeze_panes=(0, 2), sheet_name=str(w.columns[0][0]))

        workbook  = writer.book
        worksheet = writer.sheets[str(w.columns[0][0])]
        worksheet.set_column(0, len(w), 10)
        worksheet.insert_image(f'{chr(len(w.columns) + 67)}9', 'WSoccer logo.png')